In [1]:
import pandas as pd
from transformers import pipeline

In [2]:
#Cargar el df de comentarios Traducirlo?
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/df_coments_extract.csv')

In [3]:
df.head()

,Rating,Comentario,Url,Timestamp
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038


In [4]:
# Función para truncar los comentarios y manejar valores nulos
def truncate_coment(Comentario, max_length=512):
    if not isinstance(Comentario, str) or pd.isnull(Comentario):
        return "Reseña no disponible"
    return Comentario[:max_length]

In [5]:
# Función para convertir las estrellas en sentimientos
def convert_to_sentiment(label):
    star_rating = int(label[0])  # Extraer el número de estrellas del label
    if star_rating <= 2:
        return "Negativo"
    elif star_rating == 3:
        return "Neutral"
    else:
        return "Positivo"


In [6]:
# Truncar comentarios
df["truncated_coment"] = df["Comentario"].apply(truncate_coment)

In [7]:
# Cargar el modelo para análisis de sentimientos
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [8]:
# Procesar sentimientos en lotes
truncated_comments = df["truncated_coment"].tolist()  # Convertir a lista
sentiment_results = sentiment_model(truncated_comments)  # Procesar en lote

# Añadir los resultados al DataFrame
df["label"] = [res["label"] for res in sentiment_results]
df["score"] = [res["score"] for res in sentiment_results]

# Eliminamos columnas intermedias
df = df.drop(columns=["truncated_coment"])

In [9]:
df.head()

,Rating,Comentario,Url,Timestamp,label,score
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516,4 stars,0.805798
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027,5 stars,0.752465
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032,5 stars,0.636184
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035,5 stars,0.745236
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038,4 stars,0.476711


In [11]:
# Aplicar la conversión
df["sentiment_label"] = df["label"].apply(convert_to_sentiment)
df.head()

,Rating,Comentario,Url,Timestamp,label,score,sentiment_label
0,Valoración: 5 estrellas,"Todo muy bien, el lugar es nuevo moderno y lim...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.703516,4 stars,0.805798,Positivo
1,Valoración: 5 estrellas,"Excelente alojamiento, trabajo remoto y cumpli...",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.731027,5 stars,0.752465,Positivo
2,Valoración: 5 estrellas,Un lugar estupendo,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.745032,5 stars,0.636184,Positivo
3,Valoración: 5 estrellas,"totalmente nuevo, muy limpio personaluy atento",https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.759035,5 stars,0.745236,Positivo
4,Valoración: 5 estrellas,Me quedé en Blau mientras estaba en un viaje d...,https://www.airbnb.es/rooms/126566083301808951...,2024-11-08 14:47:32.773038,4 stars,0.476711,Positivo


In [12]:
df.to_csv('df_sentiment.csv', index=False)